# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load csv
vacation_data= "cities.csv"
vacation_data= pd.read_csv(vacation_data)

vacation_df= pd.DataFrame(vacation_data)
vacation_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [12]:
# use lat and lng as locations and humidity as weight
location = vacation_df[["Lat", "Lng"]].astype(float)
weight = vacation_df["Humidity"].astype(float)

#add heatmap layer to map
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(location,weights=weight,dissipating=False,max_intensity=100,point_radius = 1, opacity=1)
fig.add_layer(heat_layer)

plt.savefig("vacation_heatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
#Narrow down the cities to fit weather conditions.
#A max temperature lower than 80 degrees but higher than 70.
temp = vacation_df.loc[vacation_df["Max Temp"]<80,:]
temp2 = temp.loc[vacation_df["Max Temp"]>70,:]
temp2.head()

#Wind speed less than 10 mph.
wind = temp2.loc[temp2["Wind Speed"]<10,:]
wind

#Zero cloudiness.
cloud = wind.loc[wind["Cloudiness"]==0,:]
cloud.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
#Store into variable named hotel_df.
hotel_df = pd.DataFrame()
#Add a "Hotel Name" column to the DataFrame.
hotel = pd.DataFrame(cloud)
hotel_df['City'] = hotel['City']
hotel_df['Country'] = hotel['Country']
hotel_df['Lat'] = hotel['Lat']
hotel_df['Lng'] = hotel['Lng']
#hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng
7,opuwo,NaN,-18.06,13.84
70,ponta do sol,BR,-20.63,-46.00
88,cidreira,BR,-30.17,-50.22
138,morondava,MG,-20.30,44.28
139,vaini,IN,15.34,74.49


In [15]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 7: opuwo.
Closest hotel to opuwo at -18.06 , 13.84 is NRCS - Namibia Red Cross Society.
------------
Retrieving Results for Index 70: ponta do sol.
Closest hotel to ponta do sol at -20.63 , -46.0 is Hostel do Lago.
------------
Retrieving Results for Index 88: cidreira.
Closest hotel to cidreira at -30.17 , -50.22 is Fausto Imóveis.
------------
Retrieving Results for Index 138: morondava.
Closest hotel to morondava at -20.3 , 44.28 is Agence BNI Madagascar.
------------
Retrieving Results for Index 139: vaini.
Closest hotel to vaini at 15.34 , 74.49 is Simba Garden Restaurant and nature camp.
------------
Retrieving Results for Index 176: nador.
Closest hotel to nador at 35.17 , -2.93 is Café Titanic مقهى تايتانيك.
------------
Retrieving Results for Index 297: mogok.
Closest hotel to mogok at 22.92 , 96.51 is Guiding Star Internet Cafe.
------------
Retrieving Results for Index 444: birjand.
Closest hotel to birjand at 32.86 , 59.22 is نمایندگی ایران خودر

In [16]:
#Store the first Hotel result into the DataFrame.
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
7,opuwo,NaN,-18.06,13.84,NRCS - Namibia Red Cross Society
70,ponta do sol,BR,-20.63,-46.00,Hostel do Lago
88,cidreira,BR,-30.17,-50.22,Fausto Imóveis
138,morondava,MG,-20.30,44.28,Agence BNI Madagascar
139,vaini,IN,15.34,74.49,Simba Garden Restaurant and nature camp


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
location = hotel_df[["Lat", "Lng"]]

In [18]:
#Plot markers on top of the heatmap.
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

weight = vacation_df["Humidity"].astype(float)
marker_layer = gmaps.marker_layer(location, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("vacationhotel_heatmap.png")

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>